# Homework Assignment 1
### [The Art of Analyzing Big Data - The Data Scientist’s Toolbox](https://www.ise.bgu.ac.il/labs/fire/lectures.html)
#### By Dr. Michael Fire 

For this homework you will need to write code that analyzes real-world datasets. The code needs to be written in Python using the [sqlite3](https://docs.python.org/2/library/sqlite3.html) package. 

**Please note:** You need to answer only the questions that match your ID first digit.

# 1. Babies Names Dataset (35pt)

**Task 1 (for everyone):** Write a code that uses the  [**babies names dataset** ](https://www.kaggle.com/kaggle/us-baby-names?select=NationalNames.csv) (use NationalNames.csv) and creates a table named (Names) with the dataset data and the following columns: 'State', 'Gender', 'Name', 'Count' , and 'Year' (5pt)
**Bonus:** Load the data using a Batch INSERT SQL Query (2pt)

In [1]:
import json
import os
import pandas as pd
!pip install kaggle 
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [2]:
api_token = {"username":"amirbialer","key":"f2af62d43f0aaf0ee3bea5998634a1b7"}
!mkdir /root/.kaggle/
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

In [3]:
# Creating a dataset directory
!mkdir ./datasets
!mkdir ./datasets/us-baby-name

In [4]:
# download the dataset from Kaggle and unzip it
!kaggle datasets download kaggle/us-baby-names -f StateNames.csv -p ./datasets/us-baby-name/
!unzip ./datasets/us-baby-name/*.zip  -d ./datasets/us-baby-name/


 76% 23.0M/30.5M [00:00<00:00, 60.2MB/s]
100% 30.5M/30.5M [00:00<00:00, 121MB/s] 
Archive:  ./datasets/us-baby-name/StateNames.csv.zip
  inflating: ./datasets/us-baby-name/StateNames.csv  


In [5]:
PATH = './datasets/us-baby-name/'
conn = sqlite3.connect(f'{PATH}/Amir_DataBase.db') 

df = pd.read_csv(f'{PATH}/StateNames.csv')
df.to_sql('StateNames', conn, index=False, if_exists='replace')

In [6]:
#Create table
c = conn.cursor()
c.execute('''CREATE TABLE IF NOT EXISTS Names
         ( 'State', 'Gender', 'Name', 'Count' , 'Year');''')
c.close()

In [7]:
#Batch insert data to Table
c = conn.cursor()
c.execute('''INSERT INTO Names ("State", "Gender", "Name", "Count" , "Year") SELECT "State", "Gender", "Name", "Count", "Year" FROM "StateNames";''')
conn.commit()
c.close()

**Task 2 (for everyone):** Write a query that returns the statistics for the name Mary (5pt). Use the [the timeit package](https://docs.python.org/3/library/timeit.html) to measure the time it takes the query to run (5pt). **Bonus:** [Create an index](https://www.w3schools.com/sql/sql_create_index.asp)  on the _Name_ column and use the [the timeit package](https://docs.python.org/3/library/timeit.html) to measure the time it takes the query to run with the index (5pt)

In [5]:
import timeit

In [9]:
def Get_Marys_Statistics():
  c = conn.cursor()
  print("Number of Marys in data: {}".format(c.execute(''' SELECT SUM(COUNT) Name FROM Names Where Name="Mary"''').fetchone()))
  print("Number of States that have a Mary: {}".format(c.execute(''' SELECT COUNT(DISTINCT(State)) FROM Names Where Name="Mary"''').fetchone()))
  s,n=c.execute(''' SELECT State, Sum(Count) FROM Names WHERE Name="Mary" Group by State
             Order by Sum(Count) DESC''').fetchone()
  print("State with most Marys: {}. It has {} Marys ".format(s,n))
  print("")
  c.close()


In [10]:
print("program takes {} seconds".format(timeit.timeit(Get_Marys_Statistics, number=1)))

Number of Marys in data: (3740495,)
Number of States that have a Mary: (51,)
State with most Marys: PA. It has 292159 Marys 

program takes 1.3925439110000042 seconds


### <span style="color:red"> Please answer only **one** of the following questions according to your ID number (use the formula **<YOUR_ID> mod 4 +1**) </span>

In [11]:
# which question to answer - put your ID number and run the code 
your_id  = "203837695"
q = int(your_id) % 4 + 1
print("You need to answer question number %s" % q)

You need to answer question number 4


***Question 1:*** Write a function that returns how many babies were born in a given state in a given year.
Use it to calculate the number of babies born in TX in 1986 (10pt)

***Question 2:*** Write a function that returns how many female babies were born between a given range of years.
Use it to calculate how many babies were born between 1988 and 1995  (10pt)

**Question 3:** Write a function that returns the most common female name in a given state. Use it to calculate the most common female name in Wasington in 1987 (10pt)

**Question 4:** Write a function that returns how many male babies named _Robert where born in a given state in a given year. Use it to find the state in which the highest number of babies _Robert_ where born in 2010 (10pt)

In [12]:
def Count_Roberts(state, year):
  c = conn.cursor()
  s= c.execute(f''' SELECT "Count" FROM Names Where Name="Robert" AND Gender="M" AND Year={year} AND State={state}''').fetchone()
  c.close()
  return s[0]

Count_Roberts('"WA"', 2010)

122

In [15]:
def Most_Roberts(year):
  c = conn.cursor()
  states= c.execute(f''' SELECT DISTINCT "State" FROM Names''').fetchall()
  df=pd.DataFrame()
  for state in states:
    cc=Count_Roberts("'"+state[0]+"'",year)
    df=df.append({"State":state, "number":cc}, ignore_index=True)
  df.sort_values(by="number", ascending=False, inplace=True)
  print(df.iloc[0])
  c.close()

Most_Roberts(2010)

State     (CA,)
number      852
Name: 4, dtype: object


**Question (for everyone):** For the state of CA write code that calculates the second most popular female/male names in each decade (10pt). **Bonus**: Visualize it somehow using Matplotlib (5pt)

In [16]:
def Second_Most_Popular():
  c = conn.cursor()
  male= c.execute(f''' SELECT dec, Name, Gender, MAX(totcount) as totcount FROM (
  SELECT CAST(Year/10 as int)*10 as dec, Name, Gender, SUM(Count) as totcount FROM Names 
                                WHERE State="CA" AND Gender="M"
                                GROUP BY dec, Name
                                ORDER BY dec, totcount DESC) as al
  GROUP BY dec
  ORDER BY dec, totcount DESC ;''').fetchall()
  female= c.execute(f''' SELECT dec, Name, Gender, MAX(totcount) as totcount FROM (
  SELECT CAST(Year/10 as int)*10 as dec, Name, Gender, SUM(Count) as totcount FROM Names 
                                WHERE State="CA" AND Gender="F"
                                GROUP BY dec, Name
                                ORDER BY dec, totcount DESC) as al
  GROUP BY dec
  ORDER BY dec, totcount DESC ;''').fetchall()
  print(male)
  print(female)

  c.close()
  
Second_Most_Popular()



[(1910, 'John', 'M', 7826), (1920, 'Robert', 'M', 22946), (1930, 'Robert', 'M', 24980), (1940, 'Robert', 'M', 48310), (1950, 'Michael', 'M', 74138), (1960, 'Michael', 'M', 78987), (1970, 'Michael', 'M', 65112), (1980, 'Michael', 'M', 73614), (1990, 'Michael', 'M', 54727), (2000, 'Daniel', 'M', 39578), (2010, 'Jacob', 'M', 15054)]
[(1910, 'Mary', 'F', 8270), (1920, 'Mary', 'F', 17696), (1930, 'Mary', 'F', 16347), (1940, 'Linda', 'F', 38380), (1950, 'Linda', 'F', 40841), (1960, 'Lisa', 'F', 43742), (1970, 'Jennifer', 'F', 54734), (1980, 'Jessica', 'F', 51796), (1990, 'Jessica', 'F', 47806), (2000, 'Emily', 'F', 30395), (2010, 'Sophia', 'F', 17166)]


# 2. Flavors of Cacao Dataset (15pt)

Using the [Flavors of Cacao](https://www.kaggle.com/rombikuboktaeder/choco-flavors) dataset, answer the following questions:

In [21]:
!mkdir ./datasets/choco_flavors
!kaggle datasets download -d rombikuboktaeder/choco-flavors -f flavors_of_cacao.csv -p ./datasets/choco_flavors/
#!unzip ./datasets/flavors_of_cacao/*.zip  -d ./datasets/flavors_of_cacao/

mkdir: cannot create directory ‘./datasets/choco_flavors’: File exists
  0% 0.00/125k [00:00<?, ?B/s]
100% 125k/125k [00:00<00:00, 39.5MB/s]


In [22]:
PATH = './datasets/choco_flavors/'
conn = sqlite3.connect(f'{PATH}/Amir_DataBase.db') 

df = pd.read_csv(f'{PATH}/flavors_of_cacao.csv')
df.to_sql('cocao', conn, index=False, if_exists='replace')

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [24]:
print(len(df))
df.iloc[:5]

1795


,Company \n(Maker-if known),Specific Bean Origin\nor Bar Name,REF,Review\nDate,Cocoa\nPercent,Company\nLocation,Rating,Bean\nType,Broad Bean\nOrigin
0,A. Morin,Agua Grande,1876,2016,63%,France,3.75,,Sao Tome
1,A. Morin,Kpime,1676,2015,70%,France,2.75,,Togo
2,A. Morin,Atsane,1676,2015,70%,France,3.00,,Togo
3,A. Morin,Akata,1680,2015,70%,France,3.50,,Togo
4,A. Morin,Quilla,1704,2015,70%,France,3.50,,Peru


In [81]:
df.columns[0]

'Company\xa0\n(Maker-if known)'

In [87]:
df['Company\xa0\n(Maker-if known)']

0       A. Morin
1       A. Morin
2       A. Morin
3       A. Morin
4       A. Morin
          ...   
1790      Zotter
1791      Zotter
1792      Zotter
1793      Zotter
1794      Zotter
Name: Company \n(Maker-if known), Length: 1795, dtype: object

**Question 1:** Write a function that returns the number of bars manufactured where the bars' BroadBean Origin is a given country. Use the function to calculate the number of bars where BroadBean Origin is 'Fiji' (15pt)

In [31]:
def Count_Bars_From(place):
  c = conn.cursor()
  res=c.execute(f'''SELECT SUM(REF) as count FROM cocao WHERE "Broad Bean\nOrigin" = {place};''').fetchone()
  c.close()
  return res

Count_Bars_From('"Fiji"')

(8898,)

**Question 2:** Write a function that returns the maximal and average cocoa percentage in a bar manufactured by a company in a specific country. Use the function to calculate the minimal and average cocoa percentage in bars manufactured by a Swiss company (15pt).

In [46]:
def Max_And_Average(country):
  c = conn.cursor()
  m=c.execute(f'''SELECT MAX(CAST(replace("Cocoa\nPercent", '%', '') as int)) as per FROM cocao WHERE "Company\nLocation" = {country};''').fetchone()[0]
  a=c.execute(f'''SELECT AVG(CAST(replace("Cocoa\nPercent", '%', '') as int)) as per FROM cocao WHERE "Company\nLocation" = {country};''').fetchone()[0]
  res={"max":m, "average":a}
  c.close()
  return res

Max_And_Average('"Switzerland"')

{'average': 71.21052631578948, 'max': 88}

**Question 3:** Calculate the second most common bean type(s) and the most rare bean type(s) (15
pt)

In [54]:
def Common_And_Rare():
  c = conn.cursor()
  second_most_common=c.execute(f'''SELECT "Bean\nType", SUM(REF) as totcount FROM cocao GROUP BY  "Bean\nType"  ORDER BY totcount DESC;''').fetchall()[1]
  most_rare=c.execute(f'''SELECT "Bean\nType", SUM(REF) as totcount FROM cocao GROUP BY  "Bean\nType"  ORDER BY totcount;''').fetchone()

  res={"second_most_common":second_most_common, "most_rare":most_rare}
  c.close()
  return res

Common_And_Rare()

{'most_rare': ('Criollo (Wild)', 56),
 'second_most_common': ('Trinitario', 429646)}

**Question 4:** Calculate the number of reviews and the average rating in each year. Calculate the number of reviews and the average rating of each company in each year (15pt)

In [63]:
def Average_Reviews():
  c = conn.cursor()
  number_reviews=c.execute(f'''SELECT "Review\nDate", SUM(REF) as totcount FROM cocao GROUP BY  "Review\nDate"  ORDER BY "Review\nDate";''').fetchall()
  average_rating=c.execute(f'''SELECT "Review\nDate", AVG(Rating) as totcount FROM cocao GROUP BY  "Review\nDate"  ORDER BY "Review\nDate";''').fetchall()

  res={"number_reviews":number_reviews, "average_rating":average_rating}

  c.close()
  return res

Average_Reviews()

{'average_rating': [(2006, 3.125),
  (2007, 3.1623376623376624),
  (2008, 2.9946236559139785),
  (2009, 3.073170731707317),
  (2010, 3.1486486486486487),
  (2011, 3.256060606060606),
  (2012, 3.178205128205128),
  (2013, 3.1970108695652173),
  (2014, 3.1892712550607287),
  (2015, 3.2464912280701754),
  (2016, 3.2260273972602738),
  (2017, 3.3125)],
 'number_reviews': [(2006, 3915),
  (2007, 11639),
  (2008, 23816),
  (2009, 47919),
  (2010, 59974),
  (2011, 116125),
  (2012, 175635),
  (2013, 202612),
  (2014, 325834),
  (2015, 446932),
  (2016, 398464),
  (2017, 46584)]}

In [90]:
def Average_Reviews_Per_Company(show_amount=10):
  c = conn.cursor()
  number_reviews=c.execute(f'''SELECT "Review\nDate","Company\xa0\n(Maker-if known)", SUM(REF) as totcount FROM cocao GROUP BY  "Review\nDate", "Company\xa0\n(Maker-if known)"  ORDER BY "Review\nDate";''').fetchall()[:show_amount]
  average_rating=c.execute(f'''SELECT "Review\nDate", "Company\xa0\n(Maker-if known)", AVG(Rating) as totcount FROM cocao GROUP BY  "Review\nDate", "Company\xa0\n(Maker-if known)"  ORDER BY "Review\nDate";''').fetchall()[:show_amount]

  res={"number_reviews":number_reviews, "average_rating":average_rating}

  c.close()
  return res

Average_Reviews_Per_Company()

{'average_rating': [(2006, 'Amedei', 4.5),
  (2006, 'Bonnat', 3.4642857142857144),
  (2006, "Cote d' Or (Kraft)", 1.0),
  (2006, 'Dolfin (Belcolade)', 1.5),
  (2006, 'Domori', 3.25),
  (2006, 'El Rey', 2.75),
  (2006, 'Felchlin', 3.0833333333333335),
  (2006, "Green & Black's (ICAM)", 2.5),
  (2006, 'Guittard', 3.1666666666666665),
  (2006, 'Jacque Torres', 2.0)],
 'number_reviews': [(2006, 'Amedei', 115),
  (2006, 'Bonnat', 422),
  (2006, "Cote d' Or (Kraft)", 48),
  (2006, 'Dolfin (Belcolade)', 63),
  (2006, 'Domori', 261),
  (2006, 'El Rey', 32),
  (2006, 'Felchlin', 410),
  (2006, "Green & Black's (ICAM)", 15),
  (2006, 'Guittard', 474),
  (2006, 'Jacque Torres', 5)]}

# 3. Kickstarter Projects Dataset (25pt)

Using the [Kickstarter Projects Dataset](https://www.kaggle.com/kemical/kickstarter-projects#ks-projects-201801.csv), answer the following questions:

**Task 1 (for everyone):** Load the dataset to SQLite DB using [PonyORM](https://ponyorm.org) (10pt)

In [4]:
!mkdir ./datasets/kickstarter
!kaggle datasets download -d kemical/kickstarter-projects -f ks-projects-201801.csv -p ./datasets/kickstarter/
!unzip ./datasets/kickstarter/*.zip  -d ./datasets/kickstarter/

 44% 9.00M/20.4M [00:00<00:00, 41.3MB/s]
100% 20.4M/20.4M [00:00<00:00, 67.8MB/s]
Archive:  ./datasets/kickstarter/ks-projects-201801.csv.zip
  inflating: ./datasets/kickstarter/ks-projects-201801.csv  


In [5]:
!pip install pony

     |████████████████████████████████| 290 kB 5.3 MB/s 
  Created wheel for pony: filename=pony-0.7.14-py3-none-any.whl size=351293 sha256=57d1b7652b98a078b8d537ebc7d0d8a9c20955d2b1d62a1af5b01f39eccdd4d8
  Stored in directory: /root/.cache/pip/wheels/de/7a/8b/82a3b5a0135fc6bad188f62e1bf6c4843142bc22c144879014
Successfully built pony


In [6]:
from pony.orm import *
import os
# Creating a new database
db = Database()
db.bind(provider='sqlite', filename=f'{os.getcwd()}/datasets/kickstarter/kickstarter.pony.db', create_db=True) # notice we can easily use other type of databases instead of SQLite


### <span style="color:red"> Please answer only **one** of the following questions according to your ID number (use the formula **<YOUR_ID> mod 3 +1**) </span>

In [7]:
# which question to answer - put your ID number and run the code 
your_id  = "203837695"
q = int(your_id) % 3 + 1
print("You need to answer question number %s" % q)

You need to answer question number 2


**Question 1:** On average which project category received the highest number of backers? (15 pt)

**Question 2:** On average which project category received the highest pledged USD? (15 pt)

In [8]:
 class Category(db.Entity):
   name=Required(str)
   main_category=Required(str)
   projects = Set('Project')

 class Project(db.Entity):
    name = Optional(str) # cannot be None
    category = Required(Category)
    usd_pledged_real = Required(float)


In [9]:
set_sql_debug(True) # helps to see what SQL commands are running
db.generate_mapping(create_tables=True) # create tables


GET CONNECTION FROM THE LOCAL POOL
PRAGMA foreign_keys = false
BEGIN IMMEDIATE TRANSACTION
CREATE TABLE "Category" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT NOT NULL,
  "main_category" TEXT NOT NULL
)

CREATE TABLE "Project" (
  "id" INTEGER PRIMARY KEY AUTOINCREMENT,
  "name" TEXT NOT NULL,
  "category" INTEGER NOT NULL REFERENCES "Category" ("id") ON DELETE CASCADE,
  "usd_pledged_real" REAL NOT NULL
)

CREATE INDEX "idx_project__category" ON "Project" ("category")

SELECT "Category"."id", "Category"."name", "Category"."main_category"
FROM "Category" "Category"
WHERE 0 = 1

SELECT "Project"."id", "Project"."name", "Project"."category", "Project"."usd_pledged_real"
FROM "Project" "Project"
WHERE 0 = 1

COMMIT
PRAGMA foreign_keys = true
CLOSE CONNECTION


In [13]:
df=pd.read_csv("/content/datasets/kickstarter/ks-projects-201801.csv")
df["category"]=df["category"].astype(str)
df["usd_pledged_real"]=df["usd_pledged_real"].astype(float)

In [14]:
for idx, row in df.iterrows():
    Project(name=row['name'] if type(row['name']) == str else '',
            category=row["category"],
            usd_pledged_real=row['usd_pledged_real'])

ValueError: ignored

In [114]:
conn = sqlite3.connect(f'{os.getcwd()}/datasets/kickstarter/kickstarter.pony.db') 


In [115]:
c=conn.cursor()
res=c.execute('''SELECT "Catagory", MAX(usd_pledged_real) as usd FROM "Project" GROUP BY "Catagory" ORDER by usd DESC ''').fetchall()[:5]
c.close()
res

[]

**Question 3:** In which month occurred the highest number of projects? (15 pt)

## 4. Oscars Datasets (10pt)

Using the [Oscars Dataset](https://www.kaggle.com/theacademy/academy-awards), please answer only one of the following questions (you can chose):

**Question 1:** Who is the male actress with the most Oscar nominees? (10pt)

**Question 2:** Who is the female director with the most Oscar nominees? (10pt)

**Question 3:** Which top-10 movies received the highest number of Oscar nominees? (10pt)

In [26]:
!mkdir ./datasets/oscar
!kaggle datasets download -d theacademy/academy-awards -f database.csv -p ./datasets/oscar/

df=pd.read_csv("/content/datasets/oscar/database.csv")
conn = sqlite3.connect(f'/content/datasets/oscar/Amir_DataBase.db') 
df.to_sql('osc', conn, index=False, if_exists='replace')
c=conn.cursor()
res=c.execute('''SELECT COUNT("Film") as count_film, "Film" FROM osc GROUP BY "Film" ORDER BY count_film DESC;''').fetchall()[:10]
c.close()
res

mkdir: cannot create directory ‘./datasets/oscar’: File exists
database.csv: Skipping, found more recently modified local copy (use --force to force download)


[(60, 'Metro-Goldwyn-Mayer'),
 (57, 'Walt Disney, Producer'),
 (42, 'John Williams'),
 (42, 'Warner Brothers'),
 (36, 'France'),
 (34, 'Alfred Newman'),
 (27, 'Italy'),
 (24, 'Paramount'),
 (22, 'Edith Head'),
 (22, 'Gordon Hollingshead, Producer')]

**Question 4:** Write a function that receives an actor's name and returns the actor’s number of Oscar nominees. Use the function to calculate the number of times Leonardo DiCaprio was a nominee (10pt)

## 5. Select a Dataset (15pt)

**Open Question:** Select an interesting dataset and use SQL to discover something interesting (15pt). **Bonus:** Use BigQuery (2pt)

In [29]:
!mkdir ./datasets/spotify
!kaggle datasets download -d pavan9065/top-100-most-streamed-songs-on-spotify -f "Top 100 most Streamed - Sheet1.csv" -p ./datasets/spotify/
df=pd.read_csv("/content/datasets/spotify/Top%20100%20most%20Streamed%20-%20Sheet1.csv")
conn = sqlite3.connect(f'/content/datasets/spotify/Amir_DataBase.db') 
df.to_sql('spot', conn, index=False, if_exists='replace')

mkdir: cannot create directory ‘./datasets/spotify’: File exists
Top%20100%20most%20Streamed%20-%20Sheet1.csv: Skipping, found more recently modified local copy (use --force to force download)


/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [30]:
df.iloc[:2]

,title,artist,top genre,year,beats.per.minute,energy,danceability,loudness.dB,liveness,valance,length,acousticness,speechiness,popularity
0,Blinding Lights,The Weeknd,canadian contemporary r&b,2020,171,73,51,-6,9,33,200,0,6,91
1,Watermelon Sugar,Harry Styles,pop,2019,95,82,55,-4,34,56,174,12,5,88


In [52]:
c=conn.cursor()
artists=c.execute('''SELECT artist, COUNT(title),AVG("beats.per.minute"),AVG(energy),AVG(danceability), AVG("loudness.dB"), AVG(liveness),AVG("length"),AVG(acousticness), AVG(speechiness),AVG(popularity)    FROM spot GROUP BY artist ORDER BY COUNT(title) DESC;''').fetchall()
artists_df=pd.DataFrame.from_dict([{"artist":a[0], "beats.per.minute":a[1], "energy":a[2], "danceability":a[2], "loudness":a[3],"liveness":a[4], "valence":a[5], "length":a[6], "acousticness":a[7], "speechiness":a[8], "popularity":a[9]} for a in artists])
print("top 10 artists in the dataset")
print(artists_df.sort_values(by="popularity", ascending=False).iloc[:10])
print("")
print("Correlation between popularity of an artists and other musical features")
print(artists_df.corrwith(artists_df["popularity"]).sort_values())
c.close()

top 10 artists in the dataset
                 artist  beats.per.minute  ...  speechiness  popularity
20  5 Seconds of Summer                 1  ...     2.000000   46.000000
63         benny blanco                 1  ...    56.000000   32.000000
47           Mark Mendy                 1  ...     5.000000   29.000000
29               Eminem                 1  ...     1.000000   27.000000
41                 Lauv                 1  ...    54.000000   25.000000
44         Lil Uzi Vert                 1  ...     0.000000   23.000000
37           Juice WRLD                 1  ...    35.000000   20.000000
9         Ariana Grande                 2  ...    41.000000   20.000000
5         Justin Bieber                 3  ...    50.333333   19.666667
18         XXXTENTACION                 2  ...    36.500000   19.000000

[10 rows x 11 columns]

Correlation between popularity of an artists and other musical features
valence            -0.154442
acousticness       -0.120686
speechiness        -0.0